# Visualizations and Numerical Values of the Paper

This notebook contains the code required to reproduce all figures and numerical results presented in the paper.  
For selected visualizations, additional numerical values are reported here for reference, as they were omitted from the paper due to page constraints.

In [ ]:
# necessary imports
import sys
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Sequence, Tuple, Union

import math
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from __future__ import annotations
from shapely.geometry import LineString, Point, MultiLineString
from tueplots import axes, bundles, figsizes
from tueplots.constants.color import rgb

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

# module imports
from src.preprocess_data import preprocess_accident_data
from src.routing_graph import (
    GraphBuildConfig,
    RiskConfig,
    RoutingMonthArtifacts,
    build_routing_graph_for_month,
    build_graph_with_risk_for_month,
    verify_graph_sanity,
    NodeKey,
)
from src.routing_algorithms import (
    nearest_graph_node,
    route_stats,
    shortest_path_by,
    constrained_min_risk_route,
    run_one_od_routing,
    path_to_multiline_latlon,
)

DATA_DIR = Path("..") / "data" / "panel"

The next cell loads the data frames that were constructed through the merging procedure executed in `01_data_preparation.ipynb`.

In [ ]:
accidents_df = pd.read_csv(DATA_DIR / "berlin_bike_accidents.csv")
segments_df = gpd.read_parquet(DATA_DIR / "berlin_bike_accident_strava_risk_core_panel.parquet")
junction_df = gpd.read_parquet(DATA_DIR / "berlin_bike_accident_node_panel.parquet")

# one geometry per node_id
crossings_gdf = (
    junction_df[["node_id", "geometry"]]
    .dropna(subset=["node_id", "geometry"])
    .drop_duplicates(subset=["node_id"])
    .copy()
)
crossings_gdf = gpd.GeoDataFrame(crossings_gdf, geometry="geometry", crs=junction_df.crs)

print("segments_df:", segments_df.shape)
print("junction_df:", junction_df.shape)
print("crossings_gdf:", crossings_gdf.shape)
print("accidents_df:", accidents_df.shape)

print(list(segments_df.columns))


In [ ]:
# Example usage of run_one_od_routing
result = run_one_od_routing(
    segments_panel_gdf=segments_df,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    year=2021, month=6,
    origin_lonlat=(13.3777, 52.5163),
    dest_lonlat=(13.4541, 52.5110),
    eps=0.10,
    eta=1.0,                          
)

print(result["status"])
print(result["graph_sanity"])
if result["status"] == "ok":
    print("Shortest length stats:", result["shortest_length_stats"])
    print("Constrained min-risk stats:", result["constrained_min_risk_stats"])
    print("ΔL:", result["delta_L"], "ΔR:", result["delta_R"])

## Figure 1

The following cell defines global plotting parameters to ensure consistent visual encoding across all network and routing figures.

In [ ]:
# Base network 
BASE_LW = 0.22
BASE_ALPHA = 0.10

# Accidents 
ACC_MS = 4.0
ACC_ALPHA = 0.90

# Junctions 
JUNC_MS = 3.5
JUNC_LW = 0.40
JUNC_ALPHA = 0.25

# High-risk segments 
RISK_LW = 1.9
RISK_ALPHA = 0.95

# Routing
ROUTE_LW = 1.6
ROUTE_ALPHA = 0.95

# OD points
OD_S = 30
OD_ALPHA = 1.0


In [ ]:
# Shared viewport + month
CENTER_LAT = 52.518589
CENTER_LON = 13.376665
LAT_TOL = 2.4e-2
LON_TOL = 4.8e-2

YEAR, MONTH = 2021, 6
year, month = 2021, 6
METRIC_EPSG = 32633


def viewport_bounds(center_lat=CENTER_LAT, center_lon=CENTER_LON, lat_tol=LAT_TOL, lon_tol=LON_TOL):
    x0, x1 = center_lon - lon_tol, center_lon + lon_tol
    y0, y1 = center_lat - lat_tol, center_lat + lat_tol
    return x0, x1, y0, y1


def apply_center_zoom(ax, center_lat=CENTER_LAT, center_lon=CENTER_LON, lat_tol=LAT_TOL, lon_tol=LON_TOL):
    x0, x1, y0, y1 = viewport_bounds(center_lat, center_lon, lat_tol, lon_tol)
    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)


# Panel (a): accidents on network
def plot_panel_a(ax, segments_m, accidents_df):
    accidents_gdf = gpd.GeoDataFrame(
        accidents_df,
        geometry=gpd.points_from_xy(accidents_df["XGCSWGS84"], accidents_df["YGCSWGS84"]),
        crs="EPSG:4326",
    )
    accidents_m = accidents_gdf.query("year == @YEAR and month == @MONTH")

    segments_m.plot(
    ax=ax,
    color=rgb.tue_dark,
    linewidth=BASE_LW,
    alpha=BASE_ALPHA,
    zorder=1,
    )

    accidents_m.plot(
        ax=ax,
        color=rgb.tue_orange,
        markersize=ACC_MS,
        alpha=ACC_ALPHA,
        zorder=3,
    )


    apply_center_zoom(ax)
    ax.axis("off")


# Panel (b): exposure + junction context
def plot_panel_b(ax, segments_m, junctions_m):
    exp_candidates = [
        "monthly_strava_trips",
        "sum_strava_total_trip_count",
        "strava_trips",
        "trip_count",
    ]
    exp_col = next((c for c in exp_candidates if c in segments_m.columns), None)
    if exp_col is None:
        raise ValueError(f"No exposure column found. Tried {exp_candidates}")

    x0, x1, y0, y1 = viewport_bounds()

    segments_m.plot(
        ax=ax,
        color=rgb.tue_dark,
        linewidth=BASE_LW,
        alpha=BASE_ALPHA,
        zorder=1,
    )

    q = segments_m[exp_col].quantile(0.90)
    high_exp = segments_m.loc[segments_m[exp_col] >= q]
    high_exp.plot(
        ax=ax,
        color=rgb.tue_red,
        linewidth=RISK_LW,
        alpha=RISK_ALPHA,
        zorder=3,
    )

    j_ctx = junctions_m.cx[x0:x1, y0:y1]
    if "degree" in j_ctx.columns:
        j_ctx = j_ctx.query("degree >= 3")

     # junctions as reference circles
    j_ctx.plot(
        ax=ax,
        markersize=JUNC_MS,
        facecolors="none",
        edgecolors=rgb.tue_dark,
        linewidths=JUNC_LW,
        alpha=JUNC_ALPHA,
        zorder=2,
    )

    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)
    ax.axis("off")


# Panel (c): routing paths
from shapely.geometry import MultiLineString, LineString

def plot_panel_c(
    ax,
    *,
    segments_m,
    junctions_m,
    segments_df,
    junction_df,
    crossings_gdf,
    eps=0.10,
    eta=1.0,
    rng_seed=69,
):
    x0, x1, y0, y1 = viewport_bounds()

    # pick OD inside window
    rng = np.random.default_rng(rng_seed)
    candidates = junctions_m.cx[x0:x1, y0:y1].copy()
    if "degree" in candidates.columns:
        candidates = candidates.query("degree >= 3")
    if len(candidates) < 2:
        raise ValueError("Not enough junction candidates inside the fixed window.")

    coords = np.column_stack([candidates.geometry.x.to_numpy(), candidates.geometry.y.to_numpy()])  # (lon, lat)
    i, j = rng.choice(len(coords), size=2, replace=False)
    origin_lonlat = (float(coords[i, 0]), float(coords[i, 1]))
    dest_lonlat   = (float(coords[j, 0]), float(coords[j, 1]))
    ox, oy = origin_lonlat
    dx, dy = dest_lonlat

    # routing (builds month graph internally; should return the graph used)
    result = run_one_od_routing(
        segments_panel_gdf=segments_df,
        crossings_gdf=crossings_gdf,
        junction_panel_gdf=junction_df,
        year=YEAR,
        month=MONTH,
        origin_lonlat=origin_lonlat,
        dest_lonlat=dest_lonlat,
        eps=eps,
        eta=eta,
        metric_epsg=METRIC_EPSG,
    )

    if result.get("status") == "disconnected":
        # If disconnected, just plot the base and mark OD
        G = None
        shortest_geom = None
        safe_geom = None
    else:
        G = result.get("graph", None)
        if G is None:
            # fallback (only safe if your build parameters match run_one_od_routing)
            artifacts = build_graph_with_risk_for_month(
                segments_panel_gdf=segments_df,
                crossings_gdf=crossings_gdf,
                junction_panel_gdf=junction_df,
                year=YEAR,
                month=MONTH,
                risk_cfg=RiskConfig(eta=eta),
            )
            G = artifacts.G

        shortest_geom = path_to_multiline_latlon(
            G, result["shortest_length_path"], metric_epsg=METRIC_EPSG, choose_by="length_m"
        )
        safe_geom = path_to_multiline_latlon(
            G, result["constrained_min_risk_path"], metric_epsg=METRIC_EPSG, choose_by="risk_total"
        )

    # Base network
    segments_m.plot(
        ax=ax,
        color=rgb.tue_dark,
        linewidth=BASE_LW,
        alpha=BASE_ALPHA,
        zorder=1,
    )

    # Junction context
    j_ctx = junctions_m.cx[x0:x1, y0:y1]
    if "degree" in j_ctx.columns:
        j_ctx = j_ctx.query("degree >= 3")
    j_ctx.plot(
        ax=ax,
        markersize=JUNC_MS,
        facecolors="none",
        edgecolors=rgb.tue_dark,
        linewidths=JUNC_LW,
        alpha=JUNC_ALPHA,
        zorder=2,
    )

    # Routes
    if shortest_geom is not None:
        gpd.GeoSeries([shortest_geom], crs="EPSG:4326").plot(
            ax=ax,
            color=rgb.tue_blue,
            linewidth=ROUTE_LW,
            alpha=ROUTE_ALPHA,
            zorder=4,
        )

    if safe_geom is not None:
        gpd.GeoSeries([safe_geom], crs="EPSG:4326").plot(
            ax=ax,
            color=rgb.tue_green,
            linewidth=ROUTE_LW,
            alpha=ROUTE_ALPHA,
            zorder=5,
        )

    # Origin / destination markers
    ax.scatter([ox], [oy], s=16, marker="o", color=rgb.tue_ocre, zorder=7)
    ax.scatter([dx], [dy], s=24, marker="x", color=rgb.tue_ocre, linewidths=1.2, zorder=7)

    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)
    ax.axis("off")

    # Return routing results for reuse
    return result, G, shortest_geom, safe_geom, (ox, oy, dx, dy)


def make_three_panel_figure(
    *,
    segments_df,
    junction_df,
    accidents_df,
    crossings_gdf,
    eps=0.10,
    eta=1.0,
    outpath="../report/figs/map_3panel.pdf",
):
    segments_m  = segments_df.query("year == @YEAR and month == @MONTH").copy()
    junctions_m = junction_df.query("year == @YEAR and month == @MONTH").copy()

    with plt.rc_context(bundles.icml2024(column="full", nrows=1, ncols=3)):
        fig, axs = plt.subplots(1, 3)

        plot_panel_a(axs[0], segments_m, accidents_df)
        plot_panel_b(axs[1], segments_m, junctions_m)
        routing_result = plot_panel_c(
            axs[2],
            segments_m=segments_m,
            junctions_m=junctions_m,
            segments_df=segments_df,
            junction_df=junction_df,
            crossings_gdf=crossings_gdf,
            eps=eps,
            eta=eta,
        )

        labels = ["(a) Data", "(b) Risk estimation", "(c) Safety routing"]
        for ax, lab in zip(axs, labels):
            ax.text(0.5, -0.06, lab, transform=ax.transAxes, ha="center", va="top")

        fig.savefig(outpath)
        return fig, axs, routing_result


In [ ]:
fig, axs, (result_from_fig, G_from_fig, shortest_geom_from_fig, safe_geom_from_fig, od_coords) = make_three_panel_figure(
    segments_df=segments_df,
    junction_df=junction_df,
    accidents_df=accidents_df,
    crossings_gdf=crossings_gdf,
    eps=0.10,
    eta=1.0,
    outpath="../report/figs/map_3_panels.pdf",
)
plt.show()


### Numerical Values (omitted from the paper)

The next cell uses the routing results computed in the previous cell and unpacks the results from the `make_three_panel_figure` function.

In [ ]:
result_berlin = result_from_fig
G = G_from_fig
shortest_geom = shortest_geom_from_fig
safe_geom = safe_geom_from_fig
ox, oy, dx, dy = od_coords


The next cell computes the route statistics.

In [ ]:
# Check route lengths + detour constraint
p_len = result_berlin["shortest_length_path"]
p_safe = result_berlin["constrained_min_risk_path"]

paths_identical = (p_len == p_safe)
print("Paths identical:", paths_identical)

# Choose-by for MultiGraph edge selection:
st_len  = route_stats(G, p_len,  choose_by="length_m")
st_safe = route_stats(G, p_safe, choose_by="risk_total")

shortest_length = float(st_len["length_m"])
safe_length     = float(st_safe["length_m"])

print(f"Shortest path length: {shortest_length:.1f} m")
print(f"Safe route length:    {safe_length:.1f} m")

# Report route risk 
print(f"Shortest route risk_total: {float(st_len['risk_total_sum']):.3f}")
print(f"Safe route risk_total:     {float(st_safe['risk_total_sum']):.3f}")

# Check detour constraint
eps = result_berlin.get("params", {}).get("eps", result_berlin.get("eps", 0.10))
max_allowed = (1.0 + float(eps)) * shortest_length
detour_pct = ((safe_length / shortest_length) - 1.0) * 100.0 if shortest_length > 0 else float("nan")

print(f"Detour: {detour_pct:.2f}%")
print(f"Max allowed length: {max_allowed:.1f} m")

if safe_length <= max_allowed + 1e-6:
    print("Constraint satisfied.")
else:
    print("WARNING: Constraint violated (check routing selection / edge-choice consistency).")

# Report deltas from result_berlin for cross-check
if "delta_L" in result_berlin and result_berlin["delta_L"] is not None:
    print(f"delta_L: {100*result_berlin['delta_L']:.2f}%")
if "delta_R" in result_berlin and result_berlin["delta_R"] is not None:
    print(f"delta_R: {100*result_berlin['delta_R']:.2f}%")


## Figure 2

In [ ]:

def plot_segment_risk(
    filepath_segments: str,
    filepath_junctions: str,
    risk_column: str = "risk_estimate",
    cmap: str = "coolwarm",
    output_path: str | None = None,
    norm = LogNorm(vmin=0.1, vmax=51.0)
):
    plt.rcParams.update(bundles.icml2024())
    fig, ax = plt.subplots()

    gdf_segments = gpd.read_parquet(filepath_segments)
    gdf_junctions = gpd.read_parquet(filepath_junctions)

    if risk_column not in gdf_segments.columns:
        raise ValueError(f"Column '{risk_column}' not found.")

    gdf_segments.plot(
        ax=ax,
        column=risk_column,
        cmap=cmap,
        linewidth=0.2,
        legend=True,
        legend_kwds={
            "label": "Risk Estimate (Log scale)",
            "shrink": 0.6,
            "pad": 0.05,
        },
        norm=norm,
        zorder=1
    )

    gdf_junctions.plot(     
        ax=ax,
        column=risk_column,
        cmap=cmap,
        linewidth=0.2,
        norm=norm,
        markersize=0.1,
        zorder=2
    )

    # Hide axes
    ax.set_axis_off()
    plt.title("Relative Risk per Segment", pad=0.5)    

    if output_path:
        plt.savefig(output_path, dpi=100)
    plt.show()


plot_segment_risk(
    filepath_segments="../data/panel/segments_with_risk.parquet",
    filepath_junctions="../data/panel/junctions_with_risk.parquet",
    risk_column="rr_secure",
    cmap="coolwarm",
    output_path="../report/figs/risk_segments.pdf",
)
